<a href="https://colab.research.google.com/github/Arimoro2020/Predicting-the-Churn-of-Telecom-Users/blob/main/03_Model_Building_with_KNN%2C_Random_Forest_%26_XGBoost_Telecom_Users_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Drive**

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import Libraries**

In [85]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import recall_score

**Load data & Check first five rows**

In [57]:
telecom_df_new = pd.read_excel('/content/drive/MyDrive/PTDataScience/Project2/01_DataCleansing_Telecom_Churn.xlsx')
telecom_df_new.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No
1,9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No
2,9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes
3,6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No
4,2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No


**Set CoustomerID as new Index**

In [58]:
telecom_df_new.set_index('customerID', inplace=True)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No


**Convert nominal categorical variables to dummy variables**

In [59]:
Gender = pd.get_dummies(telecom_df_new['gender'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, Gender], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male
customerID,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1


In [60]:
senior = pd.get_dummies(telecom_df_new['SeniorCitizen'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, senior], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1
customerID,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1,0
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0,0
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1,0
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1,0


In [61]:
partner = pd.get_dummies(telecom_df_new['Partner'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, partner], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1,Yes
customerID,,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1,0,1
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0,0,0
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1,1
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1,0,0
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1,0,0


In [62]:
dependents = pd.get_dummies(telecom_df_new['Dependents'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, dependents], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1,Yes,Yes
customerID,,,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1,0,1,1
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0,0,0,0
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1,1,0
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1,0,0,0
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1,0,0,0


In [63]:
phone = pd.get_dummies(telecom_df_new['PhoneService'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, phone], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1,Yes,Yes,Yes
customerID,,,,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1,0,1,1,1
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0,0,0,0,1
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1,1,0,1
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1,0,0,0,1
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1,0,0,0,1


In [64]:
lines = pd.get_dummies(telecom_df_new['MultipleLines'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, lines], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1,Yes,Yes,Yes,No phone service,Yes
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1,0,1,1,1,0,1
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0,0,0,0,1,0,0
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1,1,0,1,0,1
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1,0,0,0,1,0,0
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1,0,0,0,1,0,0


In [65]:
internet = pd.get_dummies(telecom_df_new['InternetService'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, internet], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1,Yes,Yes,Yes,No phone service,Yes,Fiber optic,No
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1,0,1,1,1,0,1,0,1
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0,0,0,0,1,0,0,1,0
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1,1,0,1,0,1,1,0
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1,0,0,0,1,0,0,0,0
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1,0,0,0,1,0,0,0,0


In [66]:
security = pd.get_dummies(telecom_df_new['OnlineSecurity'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, security], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1,Yes,Yes,Yes,No phone service,Yes,Fiber optic,No,No internet service,Yes
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1,0,1,1,1,0,1,0,1,1,0
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0,0,0,0,1,0,0,1,0,0,0
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1,1,0,1,0,1,1,0,0,0
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1,0,0,0,1,0,0,0,0,0,0
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1,0,0,0,1,0,0,0,0,0,1


In [67]:
backup = pd.get_dummies(telecom_df_new['OnlineBackup'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, backup], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1,Yes,Yes,Yes,No phone service,Yes,Fiber optic,No,No internet service,Yes,No internet service,Yes
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1,0,1,1,1,0,1,0,1,1,0,1,0
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0,0,0,0,1,0,0,1,0,0,0,0,1
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1,1,0,1,0,1,1,0,0,0,0,0
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1,0,0,0,1,0,0,0,0,0,0,0,0
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1,0,0,0,1,0,0,0,0,0,1,0,0


In [68]:
protection = pd.get_dummies(telecom_df_new['DeviceProtection'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, protection], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1,Yes,Yes,Yes,No phone service,Yes,Fiber optic,No,No internet service,Yes,No internet service,Yes,No internet service,Yes
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1,0,1,1,1,0,1,0,1,1,0,1,0,1,0
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1


In [69]:
support = pd.get_dummies(telecom_df_new['TechSupport'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, support], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1,Yes,Yes,Yes,No phone service,Yes,Fiber optic,No,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1,0,1,1,1,0,1,0,1,1,0,1,0,1,0,1,0
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0


In [70]:
tv = pd.get_dummies(telecom_df_new['StreamingTV'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, tv], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1,Yes,Yes,Yes,No phone service,Yes,Fiber optic,No,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1,0,1,1,1,0,1,0,1,1,0,1,0,1,0,1,0,1,0
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [71]:
movies = pd.get_dummies(telecom_df_new['StreamingMovies'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, movies], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1,Yes,Yes,Yes,No phone service,Yes,Fiber optic,No,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1,0,1,1,1,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0


In [72]:
paperless = pd.get_dummies(telecom_df_new['PaperlessBilling'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, paperless], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1,Yes,Yes,Yes,No phone service,Yes,Fiber optic,No,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes,Yes
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1,0,1,1,1,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0


In [73]:
payment = pd.get_dummies(telecom_df_new['PaymentMethod'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, payment], axis=1)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1,Yes,Yes,Yes,No phone service,Yes,Fiber optic,No,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes,Yes,Credit card (automatic),Electronic check,Mailed check
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No,1,0,1,1,1,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,1,0,0
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0


**Map ordinal values to the Contract feature**

In [74]:
telecom_df_new['Contract'] = telecom_df_new['Contract'].map({'Month-to-month': 1, 'One year': 2, 'Two year': 3})
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Male,1,Yes,Yes,Yes,No phone service,Yes,Fiber optic,No,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes,No internet service,Yes,Yes,Credit card (automatic),Electronic check,Mailed check
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,3,No,Credit card (automatic),24.10,1734.65,No,1,0,1,1,1,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,1,Yes,Credit card (automatic),88.15,3973.20,No,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,1,0,0
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,1,Yes,Bank transfer (automatic),74.95,2869.85,Yes,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,1,Yes,Electronic check,55.90,238.50,No,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,1,No,Electronic check,53.45,119.50,No,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0


**Create features variable and Target variable**

In [75]:
X = telecom_df_new.drop(columns=['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'PaymentMethod', 'Churn'], axis=1).values
X.shape

(5960, 29)

In [76]:
y = telecom_df_new['Churn'].values
y.shape

(5960,)

**GridSearch the KNeighborsClassifier on Dataset for optimal hyperparameters**

In [77]:
knc = KNeighborsClassifier()
parameters = [{
    'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'brute', 'kd_tree', 'ball_tree']
    
}]
search_knc = GridSearchCV(knc, parameters, scoring='accuracy', cv=5)

result = search_knc.fit(X, y)
print(result.best_score_)
print(result.best_params_)

0.7813758389261745
{'algorithm': 'brute', 'n_neighbors': 10, 'weights': 'uniform'}


**GridSearch Random Forest Classifier on the Dataset for optimal hyperparameters**

In [78]:
rfc = RandomForestClassifier()

parameters = [{
    'n_estimators': np.linspace(1, 100, 100).astype(int),
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'criterion': ['gini', 'entropy']
}]   
search_rfc = GridSearchCV(rfc, parameters, scoring='accuracy', cv=5)

result = search_rfc.fit(X, y)
print(result.best_score_)
print(result.best_params_)

0.8060402684563759
{'criterion': 'entropy', 'max_depth': 8, 'n_estimators': 79}


**GridSearch XGBoost Classifier on the Dataset for optimal hyperparameters**

In [79]:
xgbc = XGBClassifier()

parameters = [{
    'n_estimators': np.linspace(1, 100, 100).astype(int),
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'booster': ['gbtree', 'gblinear', 'dart']
}]   
search_xgbc = GridSearchCV(xgbc, parameters, scoring='accuracy', cv=5)

result = search_xgbc.fit(X, y)
print(result.best_score_)
print(result.best_params_)

0.8050335570469798
{'booster': 'gbtree', 'max_depth': 3, 'n_estimators': 95}


**Split data into training and testing data**




In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40, stratify=y)

**Standardize X_train and X_test data**

In [81]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**Use the best parameters result of the Gridsearch to create the models and evaluate them:**

In [82]:
knn = KNeighborsClassifier(algorithm='brute', n_neighbors=10, weights='uniform')
knn.fit(X_train,y_train)
print('Training accuracy:', knn.score(X_train, y_train))
print('Test accuracy:', knn.score(X_test, y_test))

Training accuracy: 0.8147171620325983
Test accuracy: 0.7589485458612976


In [83]:
rf = RandomForestClassifier(n_estimators=79, max_depth=8, criterion='entropy')
rf.fit(X_train,y_train)
print('Training accuracy:', rf.score(X_train, y_train))
print('Test accuracy:', rf.score(X_test, y_test))

Training accuracy: 0.8410834132310643
Test accuracy: 0.7835570469798657


In [90]:
xgb = XGBClassifier(n_estimators=95, max_depth=3, booster='gbtree')
xgb.fit(X_train,y_train)
print('Training accuracy:', xgb.score(X_train, y_train))
print('Test accuracy:', xgb.score(X_test, y_test))

Training accuracy: 0.8298178331735379
Test accuracy: 0.7857941834451901
